In [1]:
import pandas as pd
import os
import datetime as dt
import re
import xlsxwriter

In [4]:
class AccrualTable():
    def __init__(self, path, sheet_name, month, year):
        """
        path - путь до файла и его имя
        sheet_name - название листа в Excel
        year - год (число от 2014 до 2050)
        month - число от 1 до 12 ИЛИ если нужно сделать отчет за год, исп. параметр 'all'
        """
        if not (2014 <= year <= 2050):
            raise ValueError("Year is incorrect")
        self.path = path
        self.sheet_name = sheet_name
        self.month = month
        self.year = year
        self.final_table = self.load_excel()
        self.raw_df = self.load_excel()
        if month == "all":
            self.df = self.get_year_df()
            self.file_name = self.df['Дата заявки'].iloc[0].strftime("%Y") + "_HO.HR_BGK.xlsx"
        elif 0 < month <= 12:
            self.df = self.get_month_df()
            self.file_name = self.df['Дата заявки'].iloc[0].strftime("%Y.%m") + "_HO.HR_BGK.xlsx"
        self.df = self.calculate_without_nds()
        self.df = self.get_table_format()
    
    def load_excel(self):
        """
        Загружает данные из Excel файла
        """
        excel_file = pd.ExcelFile(self.path, engine='openpyxl')
        table = excel_file.parse(self.sheet_name)
        print(f"Loaded excel: {self.path}, {self.sheet_name}")
        return table
    
    def get_month_df(self):
        """
        Выбирает данные только за нужный месяц
        """
        mask = (self.raw_df['Дата заявки'].dt.month == self.month) & (self.raw_df['Дата заявки'].dt.year == self.year)
        print(f"Information for: {self.month}.{self.year} has been loaded")
        return self.raw_df[mask].copy()
    
    def get_year_df(self):
        """
        Выбирает данные только за нужный год
        """
        mask = self.raw_df['Дата заявки'].dt.year == self.year
        print(f"Information for: {self.year} has been loaded")
        return self.raw_df[mask].copy()
    
    
    def calculate_without_nds(self):
        """
        Рассчитывает столбец "Без НДС", если в сумму НДС не включен, значение остается прежним
        """
        self.df['with_nds'] = ~self.df['Назначение платежа'].str.lower().str.contains("без ндс")
        self.df['Без НДС'] = self.df['Сумма заявки']
        self.df.loc[self.df['with_nds'], 'Без НДС'] = round(self.df['Без НДС'] / 1.2, 2)
        print("Column Sum without NDS was added")
        return self.df
    
    def get_table_format(self):
        """
        Добавляет недостающие колонки, присваевает им нужные названия и возвращает таблицу 
        в правильном формате
        """
        self.df['Аккруал'] = 'Да'
        self.df['Юр. лицо'] = ''
        self.df['Код номенклатуры'] = 'УслА013'
        self.df['Валюта'] = 'RUB'
        self.df['Период'] = self.df['Дата заявки'].dt.strftime("%m.%Y")
        
        table_raw = self.df[['Аккруал', 'ЦФО', 'Юр. лицо', 'Вид ДДС', 'Статья ДиР', 
                             'Код номенклатуры', 'Счет', 'Код договора', 'Без НДС', 
                             'Валюта заявки', 'Валюта', 'Код подразделения', 'Дата заявки', 'Период', 'Созданные дата и время']]
        
        table_raw = table_raw.rename(columns={'Вид ДДС': 'Наименование статьи ДиР', 
                                              'Статья ДиР': 'Код статьи ДиР', 'Счет': 'Контрагент',
                                              'Без НДС': 'Сумма начисления в рублях (без НДС)',
                                              'Валюта заявки': 'Валюта суммы начисления', 'Дата заявки': 'Дата'})
        print('Your table is ready!')
        return table_raw
    
    
    def save_excel_file(self):
        """
        Сохраняет таблицу в Excel файл
        """
        with pd.ExcelWriter(self.file_name, 
                            engine='xlsxwriter',
                            datetime_format='dd.mm.yyyy') as writer:
            self.df.to_excel(writer, index=False)
            print(f"Excel file {self.file_name} was created")

In [5]:
table = AccrualTable("akkrual 28.12.xlsx", "Sheet1", 12, 2020)

Loaded excel: akkrual 28.12.xlsx, Sheet1
Loaded excel: akkrual 28.12.xlsx, Sheet1
Information for: 12.2020 has been loaded
Column Sum without NDS was added
Your table is ready!


In [44]:
table.save_excel_file()

Excel file 2020.12_HO.HR_BGK.xlsx was created


In [6]:
table_year = AccrualTable("akkrual 25.12.tmp.xlsx", "Sheet1", 'all', 2020)

Loaded excel: akkrual 25.12.tmp.xlsx, Sheet1
Loaded excel: akkrual 25.12.tmp.xlsx, Sheet1
Information for: 2020 has been loaded
Column Sum without NDS was added
Your table is ready!


In [9]:
table_year.save_excel_file()

Excel file 2020_HO.HR_BGK.xlsx was created
